![Alt text](http://www.ucm.es/logo/ucm.png "a title")

<div align="center"> 
<font size=5> Máster en Nuevas Tecnologías Electrónicas y Fotónicas </font>
</div>
    
<div align="center"> 
<font size=4> Óptica Digital, curso 2022-2023 </font>
</div>

    
<div align="center"> 
<font size=4> Ejercicio 1 - Propagación de la luz </font>
</div>

<style>
.alert-green {
  color: rgb(60,118,61) !important;
}
</style>


- **Fecha**: 11-Oct-2022
        
- **Alumno**: Alex Recuenco (Alejandro G Recuenco) 


In [ ]:
from dataclasses import dataclass
from matplotlib import pyplot as plt
from diffractio import mm, um
from diffractio.scalar_masks_XZ import Scalar_mask_XZ
from diffractio.utils_drawing import draw_several_fields


def draw_intensity_phase_XZ(field: Scalar_mask_XZ, title: str):
    # Bug: Draw_several_fields only works for XY
    # draw_several_fields(
    #   fields=(field, field),
    #   kinds=['intensity', 'phase'],
    #   **kwargs
    #   )
    field.draw("intensity")
    if title:
        plt.title(title)
    plt.show()

    field.draw("phase")
    if title:
        plt.title(title)
    plt.show()


def draw_intensity_phase_XY(field: Scalar_mask_XZ, title: str):
    # Bug: Draw_several_fields only works for XY
    draw_several_fields(fields=(field, field), kinds=["intensity", "phase"], title=title, logarithm = 1e2 )
    axes = plt.gca()
    axes.figure._suptitle.set_fontsize(12)

    plt.show()


## Problema 1

1. En el marco XZ, cree un haz gaussiano con longitud de onda $\lambda = 2\mu m$, y observe cómo varía su propagación (amplitud y fase) a medida que cambia la anchura de su cintura, para w0=25 $\mu m$ y w0=2 $\mu m$. Para la propagación utilice la función WPM.


In [ ]:
import diffractio
import numpy as np

from diffractio.scalar_sources_X import Scalar_source_X
from diffractio.scalar_masks_X import Scalar_mask_X
from diffractio.scalar_masks_XZ import Scalar_mask_XZ


@dataclass
class SimulationConfig:
    simulation_radius: float = 0.1 * mm
    simulation_distance: float = 3 * mm
    simulation_resolution: int = 1024


@dataclass
class SimulationParams:
    beam_width: float
    wavelength: float = 2 * um


def plot_gaussian_evolution(params: SimulationParams, config=SimulationConfig()):
    beam_width = params.beam_width
    wavelength = params.wavelength

    x0 = np.linspace(
        -config.simulation_radius,
        config.simulation_radius,
        config.simulation_resolution,
    )
    z0 = np.linspace(-50 * um, config.simulation_distance, config.simulation_resolution)

    u0 = Scalar_source_X(x=x0, wavelength=wavelength)
    u0.gauss_beam(0, beam_width, 0)
    u1 = Scalar_mask_XZ(x=x0, z=z0, wavelength=wavelength)
    u1.incident_field(u0)
    u1.WPM(has_edges=True)
    title = f"Gauss beam with $ w_0={beam_width/um:.0f} \mu m$"
    draw_intensity_phase_XZ(u1, title=title)


# Same simulation distance to compare their spread.
plot_gaussian_evolution(
    params=SimulationParams(beam_width=25 * um), 
    config=SimulationConfig(simulation_radius=.1 * mm, simulation_distance=100 * um),
)
plot_gaussian_evolution(
    params=SimulationParams(beam_width=2 * um),
    config=SimulationConfig(simulation_radius=25 * um, simulation_distance=100 * um),
)


### Conclusiones:

Primero, existe un problema en los dibujos de fase, creo.

Luego, cuando el width del beam es comparable al wavelength de la onda incidente, el beam se abre más rápido.

## Problema 2
1. En el marco XY, cree una máscara que esté formada por dos aberturas: 1 círculo de 50 micras de diámetro y ubicado en la posición (x,y)=(-100 um,0) y un cuadrado de 100 micras dde lado y ubicado en la posición (x,y)=(100 um, 0). Se ilumina por una onda armónica plana de  $\lambda = 0.6328\mu m$ y se obsera el campo a una distancia de 2 mm. Para la propagación utilice la función RS.


In [ ]:
from matplotlib import rcParams
rcParams['figure.dpi']=250

from typing import Tuple
from diffractio.scalar_sources_XY import Scalar_source_XY
from diffractio.scalar_masks_XY import Scalar_mask_XY

from diffractio import degrees, mm, plt, sp, um, np
from diffractio.scalar_sources_X import Scalar_source_X
from diffractio.scalar_masks_X import Scalar_mask_X


@dataclass
class SimulationConfig:
    simulation_radius: float = 0.1 * mm
    simulation_resolution: int = 1024
    amplification:int = 1


@dataclass
class SimulationParams:
    circle_radius: float = 75 * um
    circle_pos: Tuple[float, float] = (-100 * um, 0)

    square_size: float = 150 * um
    square_pos: Tuple[float, float] = (100 * um, 0)
    square_angle: float = 0

    wavelength: float = 0.6328 * um

    observation_distance: float = 2 * mm


def plot_evolution(params = SimulationParams(), config=SimulationConfig()):

    x0 = np.linspace(
        -config.simulation_radius,
        config.simulation_radius,
        config.simulation_resolution,
    )
    y0 = np.linspace(
        -config.simulation_radius,
        config.simulation_radius,
        config.simulation_resolution,
    )
    u1 = Scalar_source_XY(x=x0, y=y0, wavelength=params.wavelength)
    u1.plane_wave(A=1, theta=np.pi/2, phi=0)

    circle_mask = Scalar_mask_XY(x=x0, y=y0, wavelength=params.wavelength)
    circle_mask.circle(r0=params.circle_pos, radius=params.circle_radius)
    square_mask = Scalar_mask_XY(x=x0, y=y0, wavelength=params.wavelength)
    square_mask.square(
        r0=params.square_pos, size=params.square_size, angle=params.square_angle
    )
    total_mask = circle_mask + square_mask
    # Cast into 1s and zeros... I don't know how else to do this
    total_mask.u = (total_mask.u > 0) + 0
    total_mask.draw()
    plt.show()
    u2 = u1*total_mask

    observation_plane = u2.RS(params.observation_distance, new_field=True, amplification= (config.amplification,config.amplification), verbose=True)
    draw_intensity_phase_XY(observation_plane, title=f"Wave at {params.observation_distance/mm:.1f}mm of source. Observation window of size {config.amplification*config.simulation_radius/mm:.1f}mm")

plot_evolution(config = SimulationConfig(simulation_radius = .5*mm))
plot_evolution(config = SimulationConfig(simulation_radius = 5*mm))
plot_evolution(config = SimulationConfig(simulation_radius = 5*mm, amplification = 2))


### Conclusiones

Primero, Las figuras creo que muestran problemas en mi simulación. He debido poner algun parametro mal.

Las figuras del circulo y el rectangulo estan a una distancia suficiente para que su respuesta no solape a 2mm. 

He modificado los parametros como pide al final de los enunciados para conseguir unas distancias donde el solape sea mas visible.

## Problema 3

1. En el marco XY, vamos a utilizar la máscara square_circle que es una función entre un cuadrado y un círculo, según el parámetro s entre 0 y 1, donde s=0 es un círculo y s=1 es un cuadrado. Queremos ver la figura de difracción en campo lejano. Cree una máscara de 100 $\mu m$ de tamaño (R1=100*um, R2=100*um) y utilice una lente de focal de 2 mm y 200 um de radio para observar la figura de difracción en el plano focal de la lente. Determine la intensidad luminosa para s=0.75 y s=0.95. Para la propagación utilice la función RS.

    Tenga como referencia para los resultados el artículo M. Fernandez Guasti and M. De La Cruz Heredia, “Diffraction pattern of a circle/square aperture,” J. Mod. Opt., vol. 40, no. 6, pp. 1073–1080, 1993.
    
    Para preguntar por una función determinada, en jupyter, puede realizar lo siguiente:
    't0.square_circle?' donde t0 es una máscara definida previamente.


Proponga un valor para los parámetros que no vengan en el enunciado. En todos los casos, los resultados son gráficas de intensidad y fase luminosa.

In [ ]:
@dataclass
class SimulationConfig:
    simulation_radius: float = 0.1 * mm
    simulation_resolution: int = 1024
    amplification: int = 1


@dataclass
class SimulationParams:
    square_circle_s: float = 0.75
    square_circle_r: Tuple[float, float] = (100 * um, 100 * um)
    square_circle_pos: Tuple[float, float] = (0, 0)

    lens_pos: Tuple[float, float] = (0, 0)
    lens_radii: float = 200 * um
    focal: float = 10 * mm

    wavelength: float = 0.6328 * um

    observation_distance: float = 10 * mm


def plot_evolution(params=SimulationParams(), config=SimulationConfig()):

    x0 = np.linspace(
        -config.simulation_radius,
        config.simulation_radius,
        config.simulation_resolution,
    )
    y0 = np.linspace(
        -config.simulation_radius,
        config.simulation_radius,
        config.simulation_resolution,
    )
    u1 = Scalar_source_XY(x=x0, y=y0, wavelength=params.wavelength)
    u1.plane_wave(A=1, theta=np.pi / 2, phi=0)

    circle_square_mask = Scalar_mask_XY(x=x0, y=y0, wavelength=params.wavelength)
    circle_square_mask.square_circle(
        r0=params.square_circle_pos,
        R1=params.square_circle_r[0],
        R2=params.square_circle_r[1],
        s=params.square_circle_s,
        angle=0,
    )

    u2 = u1 * circle_square_mask

    lens_mask = Scalar_mask_XY(x=x0, y=y0, wavelength=params.wavelength)
    lens_mask.lens(r0=params.lens_pos, radius=params.lens_radii, focal=params.focal)
    u2.RS(params.observation_distance, new_field=False)

    u3=u2*lens_mask

    observation_plane = u3.RS(
        params.focal,
        new_field=True,
    )
    draw_intensity_phase_XY(
        observation_plane,
        title=f"s={params.square_circle_s:.2f}. Wave at {params.observation_distance/mm:.1f}mm of source. Observation window of size {config.amplification*config.simulation_radius/mm:.1f}mm",
    )


plot_evolution(
    config=SimulationConfig(simulation_radius=0.2 * mm),
    params=SimulationParams(square_circle_s=0.75),
)
plot_evolution(
    config=SimulationConfig(simulation_radius=0.2 * mm),
    params=SimulationParams(square_circle_s=0.95),
)


### Conclusion

